In [1]:
import skimage.io
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import cv2
from skimage import exposure, color
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
def calcular_promedio_color(imagen):

    promedio_color = imagen.mean(axis=(0,1))
    std = imagen.std(axis=(0,1))

    data = {
        'intensidad_Rojo': promedio_color[0], 'contraste_Rojo': std[0],
        'intensidad_Verde': promedio_color[1], 'contraste_Verde': std[1],
        'intensidad_Azul': promedio_color[2], 'contraste_Azul': std[2]
    }

    df = pd.DataFrame(data, index=[0])

    return df

In [3]:
def media_imagen_threshold(imagen):
    img_gray = color.rgb2gray(imagen)
    img_ampliacion_rango = exposure.rescale_intensity(img_gray, in_range='image', out_range=(0, 1))
    smoothed_image = cv2.GaussianBlur(img_ampliacion_rango, (5, 5), 0)
    imagen_umbralizada_globulos_blancos = (smoothed_image > 0.25).astype(np.uint8) * 255
    imagen_umbralizada_globulos_negros = 255 - imagen_umbralizada_globulos_blancos
    media_negro = np.mean(imagen_umbralizada_globulos_negros)
    std_negro = np.std(imagen_umbralizada_globulos_negros)
    return media_negro, std_negro

In [4]:
carpeta = 'data'
lista_dfs_res = []

for subcarpeta in os.listdir(carpeta):
    ruta_subcarpeta = os.path.join(carpeta,subcarpeta)
    resultados = []

    for nombre_archivo in os.listdir(ruta_subcarpeta):
        ruta_imagen = os.path.join(ruta_subcarpeta, nombre_archivo)

        if os.path.isfile(ruta_imagen) and nombre_archivo.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            imagen = skimage.io.imread(ruta_imagen)
            
            df_resultado = calcular_promedio_color(imagen)
            media_binario, std_binario = media_imagen_threshold(imagen) 
            df_resultado['media_binario'] = media_binario
            df_resultado['std_binario'] = std_binario
            resultados.append(df_resultado)

    df_res = pd.concat(resultados, ignore_index=True)
    if subcarpeta == 'healthy':
        df_res['label'] = 0
    else:
        df_res['label'] = 1


    lista_dfs_res.append(df_res)

df = pd.concat([lista_dfs_res[0], lista_dfs_res[1]], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
df.to_csv('dataset_features.csv', index=False)
print(df.head())

   intensidad_Rojo  contraste_Rojo  intensidad_Verde  contraste_Verde  \
0       150.369977       21.328753        141.615946        35.065781   
1       146.733711       10.955865        138.765872        13.009746   
2       119.471475        7.320871        106.681106        15.810581   
3       142.821028       10.603523        132.948831        12.185562   
4       100.795148        9.762737         90.476071        15.673630   

   intensidad_Azul  contraste_Azul  media_binario  std_binario  label  
0       148.569157       16.003733      10.909178    51.602619      1  
1       131.253735       12.633971       0.351059     9.454994      0  
2        95.094619       13.156449       0.317963     8.998853      0  
3       124.206493       11.223015       0.373681     9.754431      0  
4        85.898340       11.456872       2.984536    27.425341      0  


In [2]:
df = pd.read_csv('dataset_features.csv')

In [3]:
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


clasificador = LogisticRegression()

clasificador.fit(X_train, y_train)
pred = clasificador.predict(X_test)

accuracy = accuracy_score(y_test, pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 1.0000


c:\Users\Usuario\.conda\envs\RETO9\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [4]:
coefs = pd.Series(clasificador.coef_[0], index=X.columns)

coefs_sorted = coefs.abs().sort_values(ascending=False)

print("Variables ordenadas por su aportación al modelo:")
print(coefs_sorted)

Variables ordenadas por su aportación al modelo:
contraste_Verde     0.635677
std_binario         0.625826
intensidad_Azul     0.559704
intensidad_Verde    0.302836
contraste_Azul      0.270302
intensidad_Rojo     0.163128
contraste_Rojo      0.116455
media_binario       0.053390
dtype: float64


Prueba del modelo solo con las características más relevantes

In [5]:
X = df.drop('label', axis=1)
X = X[coefs_sorted.index[:4]]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


clasificador = RandomForestClassifier()

clasificador.fit(X_train, y_train)
pred = clasificador.predict(X_test)

accuracy = accuracy_score(y_test, pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 0.9545


Prueba del modelo con aplicación de validación cruzada

In [6]:
X = df.drop('label', axis=1)
y = df['label']

clasificador_lr = LogisticRegression()
scores_lr = cross_val_score(clasificador_lr, X, y, cv=5)  # cv=5 para 5 folds
print(f"Precisión de la regresión logística con validación cruzada: {scores_lr.mean():.4f}")

clasificador_rf = RandomForestClassifier()
scores_rf = cross_val_score(clasificador_rf, X, y, cv=5)  # cv=5 para 5 folds
print(f"Precisión del RandomForestClassifier con validación cruzada: {scores_rf.mean():.4f}")

c:\Users\Usuario\.conda\envs\RETO9\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Usuario\.conda\envs\RETO9\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

Precisión de la regresión logística con validación cruzada: 0.9909
Precisión del RandomForestClassifier con validación cruzada: 0.9719
